In [0]:
#@title Params
 

document_parser_lib_version='1.1.11'#@param {type:"string"}
charter_filename='/content/\u0413\u041F\u041D \u0423\u0441\u0442\u0430\u0432.docx'#@param {type:"string"}
# _git_branch = 'protocols-4'
_git_branch = 'charters'#@param {type:"string"}

# INIT

## pull code

In [0]:

import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from colab_support.renderer import HtmlRenderer
from legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:
lib_version = document_parser_lib_version
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from legal_docs import LegalDocument
 
from ml_tools import *

# from headers_detector import doc_features, load_model, make_headline_attention_vector
from hyperparams import HyperParameters
from protocol_parser import protocol_votes_re
 

## render tools

In [0]:
def json2html(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['words'])
  markup_vector = np.zeros(len(wordsmap))
 
  for hd in cd.headers:    
    span = hd['span']      
    markup_vector[span[0]:span[1]] += 2

  for tag in cd.attributes:
    if 'span' in  cd.attributes[tag]:
      span = cd.attributes[tag]['span']
      confidence = cd.attributes[tag]['confidence']
      markup_vector[span[0]:span[1]] += confidence
    else:
      warnings.warn(f'{tag} has no span')

  return renderer_.to_color_text(wordsmap.tokens, markup_vector)

def print_headers(d: LegalDocument):
  for p in d.paragraphs:
    print('\t --> 📂', d.substr(p.header))

## 💅 Init Embedder(s)

In [0]:
# from protocol_parser import  ProtocolPatternFactory
from tf_support.embedder_elmo import ElmoEmbedder
# from contract_patterns import ContractPatternFactory
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")

# protocols_factory = ProtocolPatternFactory(elmo_embedder)
# contracts_factory = ContractPatternFactory(elmo_embedder)

In [0]:
 
from legal_docs import tokenize_doc_into_sentences_map, ContractValue
from ml_tools import *
from parsing import ParsingContext
from patterns import *
 
from text_tools import *

# legal_docs.py
from tf_support.embedder_elmo import ElmoEmbedder
 



# New

## Patterns

In [0]:
# import pandas as pd
# pattern_names=['a', 'b', 'c']
# pattern__embeddings = [[1,0,0], [1,1,0], [1,0,1]]
# sentences_embeddings = [[1, 0, 0],  [1, 0, 1]]

# patterns_named_embeddings = pd.DataFrame (pattern__embeddings, columns=pattern_names)
 

# # for i, col in patterns_named_embeddings.iteritems():
# #   print(col.name, col.values)

# patterns_named_embeddings.columns

In [0]:
# patterns_named_embeddings

In [0]:
from enum import Enum, unique, EnumMeta
from typing import List
from structures import DisplayStringEnumMeta, CharterSubject
 
 

str_consent_patterns=[
  'принятие решений о совершении сделок',
  'принятие решения о согласии на совершение или о последующем одобрении сделок',
  'одобрение заключения изменения или расторжения какой-либо сделки Общества',
  'принятие решения о согласии на совершение или о последующем одобрении cделки стоимость которой равна или превышает'
]



## classes

In [0]:
from legal_docs import embedd_sentences, LegalDocumentExt

class CharterDocument(LegalDocumentExt):

  def __init__(self, doc: LegalDocument):
    super().__init__(doc)
    if doc is not None:
      self.__dict__ = doc.__dict__

    self.sentence_map: TextMap = None
    self.sentences_embeddings = None

    self.distances_per_sentence_pattern_dict = {}

    self.charity_tags = []
    self.org_levels = []
    self.constraint_tags = []
    self.org_level_tags = []
    

    self.margin_values: [ContractValue] = []

  def get_tags(self) -> [SemanticTag]:
    tags = []
    tags += self.charity_tags
    tags += self.org_levels
    tags += self.org_level_tags
    tags += self.constraint_tags

    for mv in self.margin_values:
      tags += mv.as_list()

    return tags


In [0]:

from embedding_tools import AbstractEmbedder
# from legal_docs import map_headlines_to_patterns
from charter_parser import embedd_charter_subject_patterns, get_charter_subj_attentions
from charter_parser import competence_headline_pattern_prefix
from contract_parser import find_value_sign_currency_attention

class CharterParser(ParsingContext):
  strs_subjects_patterns = {

    CharterSubject.Deal: [
      'принятие решений о совершении сделок'
    ],

    CharterSubject.Charity: [
      'пожертвований на политические или благотворительные цели',
      'предоставление безвозмездной финансовой помощи',
      'сделок дарения',
      'договоров спонсорского и благотворительного характера',
      'передача в безвозмездное пользование',
      'мены, дарения, безвозмездное отчуждение '
    ],

    CharterSubject.Lawsuit: [
      'о начале/урегулировании любых судебных споров и разбирательств',
      'заключении Обществом мирового соглашения по судебному делу с ценой иска '
    ],

    CharterSubject.RealEstate: [
      'стоимость отчуждаемого имущества',
      'сделки с имуществом Общества',
      'сделок ( в том числе нескольких взаимосвязанных сделок ) с имуществом Общества'
    ],

    CharterSubject.Insurance: [
      'заключение договоров страхования',
      'возобновления договоров страхования'
      'совершение сделок страхования'
    ],

    CharterSubject.Consulting: [
      'договора оказания консультационных услуг',
      'заключения агентского договора',
      'согласование договора оказания консультационных услуг или агентского договора',
      'оказания обществу информационных юридических услуг '
    ],

    CharterSubject.Other: [
      'решения о взыскании с Генерального директора убытков',
      'заключение договоров об отступном , новации и/или прощении долга , договоров об уступке права требования и переводе долга',
      'о выдаче или получении Обществом векселей , производстве по ним передаточных надписей , авалей , платежей',
      'нецелевое расходование Обществом денежных средств'
    ]

  }

  def _make_patterns(self):
    p=competence_headline_pattern_prefix #just shortcut
    comp_str_pat = []
    comp_str_pat += [[PATTERN_DELIMITER.join([p, ol.name]), ol.display_string.lower()] for ol in OrgStructuralLevel]
    comp_str_pat += [[PATTERN_DELIMITER.join([p, 'comp', 'q', ol.name]), f'к компетенции {ol.display_string} относятся следующие вопросы'.lower()] for ol in OrgStructuralLevel]
    comp_str_pat += [[PATTERN_DELIMITER.join([p, 'comp', ol.name]), f"компетенции {ol.display_string}".lower()] for ol in OrgStructuralLevel]

    _key = PATTERN_DELIMITER.join([p, 'comp', 'qr', OrgStructuralLevel.ShareholdersGeneralMeeting.name])
    comp_str_pat += [[_key,  'Компетенция Общего собрания акционеров Общества'.lower() ]]

    _key = PATTERN_DELIMITER.join([p, 'comp', 'qr', OrgStructuralLevel.BoardOfDirectors.name])
    comp_str_pat += [[_key,  'Компетенция Совета директоров Общества'.lower() ]]
    
    
    self.patterns_dict = comp_str_pat

  def __init__(self, embedder: AbstractEmbedder, elmo_embedder_default: AbstractEmbedder):
    ParsingContext.__init__(self, embedder)

    self.patterns_dict = []

    self.elmo_embedder_default: AbstractEmbedder = elmo_embedder_default

    self._make_patterns()
    patterns_texts = [p[1] for p in self.patterns_dict]

    self.patterns_embeddings = elmo_embedder_default.embedd_strings(patterns_texts)
    self.subj_patterns_embeddings = embedd_charter_subject_patterns(CharterParser.strs_subjects_patterns,
                                                                    elmo_embedder_default)

  def ebmedd(self, doc: CharterDocument):
    doc.sentence_map = tokenize_doc_into_sentences_map(doc, 200)

    ### ⚙️🔮 SENTENCES embedding
    doc.sentences_embeddings = embedd_sentences(doc.sentence_map, self.elmo_embedder_default)

    doc.distances_per_sentence_pattern_dict = calc_distances_per_pattern_dict(doc.sentences_embeddings,
                                                                              self.patterns_dict,
                                                                              self.patterns_embeddings)

  def analyse(self, charter: CharterDocument):

    #patterns_by_headers : (pattern_name, pattern_suffix, confidence, headers[header_number], doc.paragraphs[header_number])
    patterns_by_headers = self.map_charter_headlines_to_patterns(charter)

    # patterns_by_headers =  attribute_patternmatch_to_index(header_to_pattern_distances, threshold=0.6):
    # patterns_by_headers =  attribute_patternmatch_to_index(header_to_pattern_distances, threshold=0.6):
    #TODO use 
    #patterns_by_headers =  attribute_patternmatch_to_index(header_to_pattern_distances, threshold=HyperParameters.header_topic_min_confidence):


    charter.margin_values = []
    charter.constraint_tags = []
    charter.charity_tags = []
    # --------------
    # (('headline/comp/qr/ShareholdersGeneralMeeting', 16), 0.8978644013404846),
    filtered = [p_mapping for p_mapping in patterns_by_headers if p_mapping]
    
    found_org_levels=[]  
    kkk=0
    for p_mapping in filtered:   
      kkk+=1
      print('filtered', p_mapping)

      _paragraph_id = p_mapping[0][1] 
      print('_paragraph_id', _paragraph_id)
      _pattern_name = p_mapping[0][0] 
      print('_pattern_name', _pattern_name)
      paragraph = charter.paragraphs[_paragraph_id]
      confidence = p_mapping[1]
      _org_level_name = _pattern_name.split('/')[-1]      
      org_level:OrgStructuralLevel = OrgStructuralLevel[_org_level_name]
      subdoc = charter.subdoc_slice(paragraph.body.as_slice())

      parent_org_level_tag = SemanticTag(f"{org_level.name}-{kkk}", org_level.name, paragraph.body.span)
      parent_org_level_tag.confidence=confidence
      

      constraint_tags, values = self.attribute_charter_subjects(subdoc, self.subj_patterns_embeddings,
                                                                parent_org_level_tag)
      

      for value in values:
        value+=subdoc.start

      for constraint_tag in constraint_tags:
        constraint_tag.offset(subdoc.start)

      charter.margin_values += values
      charter.constraint_tags += constraint_tags


      _parent_org_level_tag_keys=[]      
      if charter.margin_values: 
        _key = parent_org_level_tag.get_key()
        if _key in _parent_org_level_tag_keys:
          parent_org_level_tag.kind= _key + f"-{len(_parent_org_level_tag_keys)}"
        charter.org_levels.append(parent_org_level_tag)
        _parent_org_level_tag_keys.append(_key)

      
      # charity_subj_av_words = subject_attentions_map[CharterSubject.Charity]['words']
      # charity_tag = find_charity_paragraphs(parent_org_level_tag, subdoc, (charity_subj_av_words + consent_words) / 2)
      # # print(charity_tag)
      # if charity_tag is not None:
      #   charter.charity_tags.append(charity_tag)

  def _rename_margin_values_tags(self, values):
   
    for value in values:     
      if value.sign.value<0:
        sfx='-max'
      elif value.sign.value>0:
        sfx='-min'
      else:
        sfx = ''

      value.parent.kind= f"constraint{sfx}"
    
    known_keys=[]
    k=0 #constraints numbering
    for value in values:
      k+=1
      if value.parent.get_key() in known_keys:
         value.parent.kind= f"{value.parent.kind}{TAG_KEY_DELIMITER}{k}"

      known_keys.append(value.parent.get_key())

  def attribute_charter_subjects(self, subdoc: LegalDocumentExt, emb_subj_patterns, parent_org_level_tag: SemanticTag):
    """
    :param subdoc:
    :param emb_subj_patterns:

          emb_subj_patterns[subj] = {
            'patterns':patterns,
            'embedding':patterns_emb
          }

    :return:
    """

    # ---------------
    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)
    values: List[ContractValue] = find_value_sign_currency_attention(subdoc, None)
    # -------------------

    # collect sentences having constraint values
    sentence_spans = []   
    for value in values:
      sentence_span = subdoc.tokens_map.sentence_at_index(value.parent.span[0], return_delimiters=True)
      if sentence_span not in sentence_spans:
        sentence_spans.append(sentence_span)
    sentence_spans = merge_colliding_spans(sentence_spans, eps=1)

    # ---
    # attribute sentences to subject
    constraint_tags = []

    i=0
    for span in sentence_spans:
      i+=1
      max_confidence = 0
      best_subject:CharterSubject = CharterSubject.Other

      for subj in subject_attentions_map.keys():
        av = subject_attentions_map[subj]['words']

        confidence_region = av[span[0]:span[1]]
        confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

        if confidence > max_confidence:
          max_confidence = confidence
          best_subject = subj

      #
      constraint_tag = SemanticTag(f'{best_subject.name}-{i}', best_subject.name, span, parent=parent_org_level_tag)
      constraint_tags.append(constraint_tag)

      # nest values
      for value in values:
        if constraint_tag.is_nested(value.parent.span):
          value.parent.set_parent_tag(constraint_tag)
        
      self._rename_margin_values_tags(values)

    return constraint_tags, values


  def map_charter_headlines_to_patterns(self, charter: LegalDocument):
    charter_parser = self
 

    p_suffices = [PATTERN_DELIMITER.join(   c[0].split(PATTERN_DELIMITER)[1:]) for c in self.patterns_dict ]

    # map = (pattern_name, pattern_suffix, confidence, headers[header_number], doc.paragraphs[header_number])
    patterns_names = [c[0] for c in charter_parser.patterns_dict]
    print('patterns_names', patterns_names)
    print(self.patterns_embeddings.shape)
    patterns_named_embeddings = pd.DataFrame(self.patterns_embeddings.T, columns=patterns_names)
    map_ = map_headlines_to_patterns(charter,
                                                patterns_named_embeddings,
                                                charter_parser.elmo_embedder_default,
                                                competence_headline_pattern_prefix,
                                                p_suffices)

    return map_



results = wp.read_doc(charter_filename)
# results = wp.read_doc('/content/ЕЮ Устав.docx')

for d in results['documents'][:1]:  # XXX
  if 'CHARTER' == d['documentType']:    
    charter = join_paragraphs(d, 'no_id')    

print_headers(charter)

In [0]:


def map_headlines_to_patterns(doc: LegalDocument,
                              patterns_named_embeddings,
                              elmo_embedder_default: AbstractEmbedder,
                              pattern_prefix: str,
                              pattern_suffixes: [str]):
  headers: [str] = doc.headers_as_sentences()

  if not headers:
    return [], []

  headers_embedding = elmo_embedder_default.embedd_strings(headers)

  header_to_pattern_distances = calc_distances_per_pattern (headers_embedding, patterns_named_embeddings)
  return attribute_patternmatch_to_index(header_to_pattern_distances)
   
charter_parser = CharterParser(elmo_embedder, elmo_embedder_default)
charter_parser.ebmedd(charter)



In [0]:
def _find_max_xy(vals):
  max_x = 0
  max_y = 0
  maxval = 0
  for x in range(vals.shape[0]):    
    found = False
    for y in range(vals.shape[1]):
      val = vals[max_x][max_y]
      if vals[x][y] > val:
        max_y=y
        max_x=x
        maxval=val

  return max_x, max_y, maxval


def attribute_patternmatch_to_index(header_to_pattern_distances_:pd.DataFrame, threshold=0.6):
  vals = header_to_pattern_distances_.values
  headers_n =  vals.shape[0]
  print('headers_n', headers_n)
   
  pairs=[]
  for __header_index in range(headers_n):

    header_index, pattern_index, maxval = _find_max_xy(vals)
    pattern_name = header_to_pattern_distances_.columns[pattern_index]
    max_pair = ((  pattern_name, header_index), maxval)          

    if maxval > threshold:     
      pairs.append(max_pair)

    vals[:][pattern_index] =-1
    vals[header_index][:] =-1

      # # used_keys.append(mk)
      # print('USED',mi)
      # print('USED mk', max_pair[0][0])
       
       
      

  # print ('attribute_patternmatch_to_index', pairs)
  # print(header_to_pattern_distances)
  return pairs
  
 
  
charter_parser.analyse(charter)


In [0]:
# globals()['header_to_pattern_distances'].T[3][:] = -1
# globals()['header_to_pattern_distances_']

  


In [0]:

renderer_.render_color_text(charter.tokens_cc, charter.get_tags_attention() , _range=(0,5) )

# Tests

### constraint_tags

In [0]:
for tag in charter.org_levels:
  print (tag.get_parent())
  print (tag)

  sub = charter [tag.as_slice()]
  renderer_.render_color_text(sub.tokens_cc, sub.get_tags_attention() , _range=(-5,5) )

### Tags

In [0]:
for tag in charter.get_tags():
  print (tag)

In [0]:
# from patterns import build_sentence_patterns
# from charter_parser import embedd_charter_subject_patterns

# emb_subj_patterns = embedd_charter_subject_patterns(strs_subjects_patterns, elmo_embedder_default)

# consent_patterns = build_sentence_patterns(str_consent_patterns, 'consent', elmo_embedder_default)
# consent_patterns_emb = elmo_embedder_default.embedd_strings(str_consent_patterns)

# ###-----debug-----
# emb_subj_patterns[CharterSubject.Insurance]['patterns']

## Find org level sections
 - find all headers spans
 - compare headers with patterns 
 - find interresting spans

In [0]:
header_slices = [p.header.as_slice() for p in charter.paragraphs]
header_slices

remap to sentences

In [0]:
header_slices_sent = charter.tokens_map.remap_slices(header_slices, charter.sentence_map)
header_slices_sent


In [0]:
section_ends = [ x[0] for x in header_slices_sent ] + [ len(charter.sentence_map) ] #+end of doc
section_ends

Available sentence pattern (names)

In [0]:
charter.distances_per_sentence_pattern_dict.keys()

In [0]:
# fi 
# av_competence_headline_pattern = max_exclusive_pattern_by_prefix(charter.distances_per_sentence_pattern_dict, competence_headline_pattern_prefix)
# av_competence_headline_pattern = relu(av_competence_headline_pattern, 0.66)

# av_interresting_headers = headers_att * av_competence_headline_pattern
# #words map
# # sections_spans_words = charter.sentence_map.remap_slices(v, charter.tokens_map)
 
# # v = charter.distances_per_sentence_pattern_dict['headline.head.directors.n']# - 0.1*charter.distances_per_sentence_pattern_dict['headline.head.directors.not'] 

# renderer_.render_color_text(charter.sentence_map.tokens, av_interresting_headers, _range=(0,1), separator='¶<br>')
# # renderer_.render_color_text(charter.tokens_cc, sections_spans_words, _range=(0,1) )

In [0]:
charter_parser.patterns_dict

## Find relevant (org struct) paragraphs

In [0]:
from doc_structure import get_tokenized_line_number
from ml_tools import calc_distances_per_pattern_dict
from legal_docs import headers_as_sentences, map_headlines_to_patterns
 
# patterns_by_headers, header_to_pattern_distances = charter_parser.map_charter_headlines_to_patterns(charter )
patterns_by_headers, header_to_pattern_distances  = charter_parser.map_charter_headlines_to_patterns(charter )
for k in patterns_by_headers :
  # if k:
  print(k)


print('\n\n headline/comp/BoardOfDirectors')
renderer_.render_color_text(headers_as_sentences(charter), header_to_pattern_distances['headline/comp/BoardOfDirectors'], _range=(0,1), separator='😱 <br>')


In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
# sns.set()

# Build data
np.random.seed(0) # seed the random number generator in order to make the run repeatable
dat = pd.DataFrame(header_to_pattern_distances)
df_labels = pd.DataFrame(header_to_pattern_distances)
 
print(df_labels)
 

In [0]:
print(dat.idxmax(axis=0))

In [0]:
# dat.drop( columns=['headline/BoardOfDirectors'], axis=1)

# dat.drop( [10])
plt.figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')

plt.suptitle("Seaborn heatmap example to display 2d percentage values", color='m')
sns.set(font_scale=0.8)

ax = sns.heatmap(dat.T,  fmt="", annot_kws={"size": 4}, vmin=0, vmax=1)
# ax.set(xlabel='x label', ylabel='y label')

plt.show()

## get_values_attention

In [0]:
from contract_parser import _find_most_relevant_paragraph, find_value_sign_currency_attention

def get_values_attention(subdoc, values:List[ContractValue]):
  numbers_attention = np.zeros(len(subdoc.tokens_map))
    
  for v in values:
    numbers_attention  [ v.value.as_slice() ] += v.value.confidence
    numbers_attention  [ v.currency.as_slice() ] += v.currency.confidence
    numbers_attention  [ v.sign.as_slice() ] += v.sign.confidence
    numbers_attention  [ v.parent.as_slice() ] += v.parent.confidence

  return numbers_attention/2

for p_mapping in patterns_by_headers:
  if p_mapping:
    paragraph =  p_mapping[4]
    org_level =  p_mapping[1]

    
    subdoc = charter.subdoc_slice(paragraph.body.as_slice())
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)

    #color 
    numbers_attention = get_values_attention(subdoc, values)
    
    for v in values:
      numbers_attention  [ v.value.as_slice() ] = 1
      numbers_attention  [ v.currency.as_slice() ] = 1
      numbers_attention  [ v.sign.as_slice() ] = 1

    print('%'*100)
    print(org_level)
    renderer_.render_color_text(subdoc.tokens_map.tokens, numbers_attention, _range=(0,1))

## charter_subjects_attribution

In [0]:
# del get_charter_subj_attentions

from legal_docs import remap_attention_vector
from contract_parser import _find_most_relevant_paragraph, find_value_sign_currency_attention
from legal_docs import remap_attention_vector, ContractValue, LegalDocumentExt
from ml_tools import *

 
from charter_parser import find_charity_paragraphs, get_charter_subj_attentions



 

###DEBUG attribute_charter_subjects -------------------


charter.margin_values = []
charter.constraint_tags=[]
# --------------
filtered = [p_mapping for p_mapping in patterns_by_headers if p_mapping]
for p_mapping in filtered:
  print('-'*40)
  
  paragraph = p_mapping[4]
  org_level_name = p_mapping[1].split('/')[-1]
  org_level = OrgStructuralLevel[org_level_name]
  print(org_level_name)
  subdoc = charter.subdoc_slice(paragraph.body.as_slice())
  parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

  constraint_tags, values = charter_parser.attribute_charter_subjects(subdoc, charter_parser.subj_patterns_embeddings,
                                                            parent_org_level_tag)
  
  charter.margin_values += values
  charter.constraint_tags += constraint_tags
  for c in constraint_tags:
    print(c)

# for p_mapping in patterns_by_headers:
#   print()
#   print()
#   if p_mapping:
#     print('p_mapping', p_mapping)
#     paragraph =  p_mapping[4]
#     org_level_name =  p_mapping[1].split('/')[-1]
#     org_level = OrgStructuralLevel[org_level_name]

#     subdoc = charter.subdoc_slice( paragraph.body.as_slice())
#     parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

#     constraint_tags, values = charter_parser.attribute_charter_subjects(subdoc, self.subj_patterns_embeddings, parent_org_level_tag)

#     print(org_level)
#     print('constraint_tags')
#     for c in constraint_tags:
#       print(c)
#     print('values')

for c in charter.margin_values:
  print(c.value)
  print ('quote:', charter.substr (c.parent))

In [0]:
charter.margin_values=[]
for p_mapping in patterns_by_headers:
  if p_mapping:
    # paragraph =  p_mapping[4]
    # org_level =  p_mapping[1]

    paragraph =  p_mapping[4]
    org_level_name =  p_mapping[1].split('/')[-1] # TODO FIX it, looks weird
    org_level = OrgStructuralLevel[org_level_name]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    parent_org_level_tag = SemanticTag(org_level.name, org_level, paragraph.body.span)

    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
    # charity_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
    #                                                             consulting_patterns,
    #                                                             consulting_patterns_emb)
    
    # print(order_patterns_distances.keys())
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)
    numbers_attention = get_values_attention(subdoc, values)

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)

    # charity_subj_av = relu(max_exclusive_pattern_by_prefix(charity_patterns_distances, 'consulting/'),0.6)
    # charity_subj_av_words = remap_attention_vector(charity_subj_av, subdoc.sentence_map, subdoc.tokens_map)
    # order_patterns_distances['order/consent']
    # renderer_.render_color_text(subdoc.sentence_map.tokens, v, _range=(0,1), separator='😱 <br>')
    
    # charity_patterns_distances_words = remap_attention_vector(v, subdoc.sentence_map, subdoc.tokens_map)

    
    
    # dd = get_charter_subj_attentions(subdoc, emb_subj_patterns)
    # charity_subj_av_words = dd[CharterSubject.Charity]['words']
    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    charity_subj_av_words = subject_attentions_map[CharterSubject.Charity]['words']
    charity_tag = find_charity_paragraphs(parent_org_level_tag, subdoc, (charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    if charity_tag is not None:
      charter.charity_tags.append(charity_tag)

 

    constraint_tags, values = attribute_charter_subjects(subdoc, emb_subj_patterns, parent_org_level_tag)
    charter.margin_values += values

    # renderer_.render_color_text(subdoc.tokens_cc, charity_subj_av_words + numbers_attention , _range=(0,2) )
    

In [0]:
renderer_.render_color_text(charter.tokens_cc, charter.get_tags_attention() , _range=(0,2) )

### Test attribution

In [0]:


for p_mapping in patterns_by_headers:
  if p_mapping:
    # paragraph =  p_mapping[4]
    # org_level =  p_mapping[1]

    paragraph =  p_mapping[4]
    org_level_name =  p_mapping[1].split('/')[-1]
    org_level=OrgStructuralLevel[org_level_name]

    
    subdoc = charter.subdoc_slice( paragraph.body.as_slice())
    
    consent_patterns_distances = calc_distances_per_pattern_dict(subdoc.sentences_embeddings,
                                                                consent_patterns,
                                                                consent_patterns_emb)
    
     
    

    consent_v = relu(max_exclusive_pattern_by_prefix(consent_patterns_distances, 'consent/'),0.5)
    consent_words = remap_attention_vector(consent_v, subdoc.sentence_map, subdoc.tokens_map)
     
     
    values:List[ContractValue] = find_value_sign_currency_attention(subdoc, None)

    #collect sentences with constraint values
    sentence_spans=[]
    for value in values:
      sentence_span = subdoc.tokens_map.sentence_at_index(value.parent.span[0], return_delimiters=True)      
      if sentence_span not in sentence_spans:
        sentence_spans.append(sentence_span) 
    # sentence_spans = merge_colliding_spans(sentence_spans, eps=1)
     


    subject_attentions_map = get_charter_subj_attentions(subdoc, emb_subj_patterns)

    # attribute sentences to subject
    
    for span in sentence_spans:
      print ( )
      print ('-'*30)
      print ('VALUE', sentence_span, subdoc.tokens_map.text_range(span))
      for subj in subject_attentions_map.keys():
        av = subject_attentions_map[subj]['words']
        
        confidence_region = av[span[0]:span[1]]
        confidence = estimate_confidence_by_mean_top_non_zeros(confidence_region)

        renderer_.render_color_text(subdoc.tokens_map[span[0]:span[1]], confidence_region , _range=(0,1) )
        print(subj,  confidence )
      print ('~'*30)
    numbers_attention = get_values_attention(subdoc, values)


    

    charity_subj_av_words = subject_attentions_map[CharterSubject.Consulting]['words']
    charity_tag = find_charity_paragraphs(org_level, subdoc,(charity_subj_av_words+consent_words)/2)
    print(charity_tag)
    
    



# Save json

In [0]:
json_str=charter.to_json()

json_obj = DocumentJson.from_json(json_str)
with open('charter.json', 'w') as file: 
  file.write(json_str)